In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/public_assault/dataset/violence.zip -d .
!unzip /content/drive/MyDrive/public_assault/dataset/handclapping.zip -d handclapping
!unzip /content/drive/MyDrive/public_assault/dataset/handwaving.zip  -d handwaving
!unzip /content/drive/MyDrive/public_assault/dataset/jogging.zip   -d jogging
!unzip /content/drive/MyDrive/public_assault/dataset/running.zip  -d running
!unzip /content/drive/MyDrive/public_assault/dataset/walking.zip  -d walking

Archive:  /content/drive/MyDrive/public_assault/dataset/violence.zip
   creating: ./violence/
  inflating: ./violence/V_925.mp4    
  inflating: ./violence/V_953.mp4    
  inflating: ./violence/V_240.mp4    
  inflating: ./violence/V_353.mp4    
  inflating: ./violence/V_374.mp4    
  inflating: ./violence/V_21.mp4     
  inflating: ./violence/V_290.mp4    
  inflating: ./violence/V_234.mp4    
  inflating: ./violence/V_365.mp4    
  inflating: ./violence/V_285.mp4    
  inflating: ./violence/V_945.mp4    
  inflating: ./violence/V_236.mp4    
  inflating: ./violence/V_864.mp4    
  inflating: ./violence/V_241.mp4    
  inflating: ./violence/V_257.mp4    
  inflating: ./violence/V_210.mp4    
  inflating: ./violence/V_176.mp4    
  inflating: ./violence/V_865.mp4    
  inflating: ./violence/V_230.mp4    
  inflating: ./violence/V_235.mp4    
  inflating: ./violence/V_340.mp4    
  inflating: ./violence/V_767.mp4    
  inflating: ./violence/V_348.mp4    
  inflating: ./violence/V_773.mp

In [ ]:
!pip install imageio-ffmpeg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import cv2
import os
import pickle
import PIL
import re
from PIL import Image
import imageio
from skimage.transform import resize
from keras.models import load_model

In [ ]:
CATEGORIES = [
    "violence",
    "handclapping",
    "handwaving",
    "jogging",
    "running",
    "walking"
]
IMG_W_ORIGINAL = 120
IMG_H_ORIGINAL = 160
IMG_W = 60
IMG_H = 80
CHANNELS = 1
SEQ_LEN = 30
SPACIALNET_INPUT_SHAPE = (SEQ_LEN, IMG_W, IMG_H, CHANNELS)
TEMPORALNET_INPUT_SHAPE = (SEQ_LEN-1, IMG_W//2, IMG_H//2, CHANNELS)
TEMPORALNET_INPUT_SHAPE_2x = (2 * (SEQ_LEN-1), IMG_W//2, IMG_H//2, CHANNELS)
NCLASSES = 6

In [ ]:
Train_Id = [1,4,6,8,10,11,13,16,18,19,21,24]
Validation_Id = [2,5,9,14,15,17,20,23]
Test_Id = [3,7,12,22,25]

In [ ]:
def clean_sequence_file():
    print("Cleaning dataset/00sequences.txt ...")

    # Read 00sequences.txt file.
    with open("/content/drive/MyDrive/public_assault/00sequences.txt", 'r') as content_file:
      for _ in range(20): # Skiping first 20 lines of sequences file (instruction details).
        next(content_file)
      content = content_file.read()

    # Replace tab and newline character with space, then split file's content
    # into strings.
    content = re.sub("[\t\n]", " ", content).split()

    # Dictionary to keep ranges of frames with humans.
    # Example:
    # video "person01_boxing_d1": [(1, 95), (96, 185), (186, 245), (246, 360)].
    frames_idx = {}

    # Current video that we are parsing.
    current_filename = ""

    for s in content:
        if s == "frames":
            # Ignore this token.
            continue
        elif s.find("-") >= 0:
            # This is the token we are looking for. e.g. 1-95.
            if s[len(s) - 1] == ',':
                # Remove comma.
                s = s[:-1]

            # Split into 2 numbers => [1, 95]
            idx = s.split("-")

            # Add to dictionary.
            if not current_filename in frames_idx:
                frames_idx[current_filename] = []
            frames_idx[current_filename].append((int(idx[0]), int(idx[1])))
        else:
            # Parse next file.
            current_filename = s + "_uncomp.avi"

    return frames_idx


In [ ]:
def prepare_dataset(dataset="train", sequences=None):
    if dataset == "train":
        ID = Train_Id
    elif dataset == "dev":
        ID = Validation_Id
    else:
        ID = Test_Id

    if sequences == None:
      frames_idx = clean_sequence_file()
    else:
      frames_idx = sequences

    data = []
    for category in CATEGORIES:
        if category == "violence":
            # Get all files in current category's folder.
            folder_path = os.path.join("/content/", category)
            filenames = sorted(os.listdir(folder_path))

            for filename in filenames:
                if filename == ".ipynb_checkpoints":
                    continue
                filepath ="/content/"+category+"/"+filename

                vid = imageio.get_reader(filepath, "ffmpeg")

                frames = []

                # Add each frame to correct list.
                for i, frame in enumerate(vid):
                    # Boolean flag to check if current frame contains human.
                    frame = cv2.resize(frame, (120, 160))


                    # Convert to grayscale.

                    frame = Image.fromarray(np.array(frame))
                    frame = frame.convert("L")
                    frame = np.array(frame.getdata(),
                                    dtype=np.uint8).reshape((120, 160))
                    frame = resize(frame, (60, 80))

                    frames.append(frame)

                data.append({
                    "filename": filename,
                    "category": category,
                    "frames": frames
            })
        else:
             # Get all files in current category's folder.
            folder_path = os.path.join("/content/", category)
            filenames = sorted(os.listdir(folder_path))

            for filename in filenames:
                filepath ="/content/"+category+"/"+filename

                # Get id of person in this video.
                person_id = int(filename.split("_")[0][6:])
                if person_id not in ID:
                    continue

                vid = imageio.get_reader(filepath, "ffmpeg")

                frames = []

                # Add each frame to correct list.
                for i, frame in enumerate(vid):
                    # Boolean flag to check if current frame contains human.
                    ok = False
                    for seg in frames_idx[filename]:
                        if i >= seg[0] and i <= seg[1]:
                            ok = True
                            break
                    if not ok:
                        continue

                    # Convert to grayscale.
                    frame = Image.fromarray(np.array(frame))
                    frame = frame.convert("L")
                    frame = np.array(frame.getdata(),
                                    dtype=np.uint8).reshape((120, 160))
                    frame = resize(frame, (60, 80))

                    frames.append(frame)

                data.append({
                    "filename": filename,
                    "category": category,
                    "frames": frames
            })


    pickle.dump(data, open("/content/drive/MyDrive/public_assault/pixel_feature/%s.p" % dataset, "wb+"))


In [ ]:
extracted_sequences = clean_sequence_file()
print("Preparing train dataset ...")
prepare_dataset(dataset="train", sequences=extracted_sequences)
print("Preparing dev dataset ...")
prepare_dataset(dataset="dev", sequences=extracted_sequences)
print("Preparing test dataset ...")
prepare_dataset(dataset="test", sequences=extracted_sequences)

Cleaning dataset/00sequences.txt ...
Preparing train dataset ...


<ipython-input-8-d31941a92be2>:33: DeprecationWarning:     `imresize` is deprecated!
    `imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
    Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.
  frame = imresize(frame, (120, 160))
<ipython-input-8-d31941a92be2>:42: DeprecationWarning:     `imresize` is deprecated!
    `imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
    Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.
  frame = imresize(frame, (60, 80))
<ipython-input-8-d31941a92be2>:84: DeprecationWarning:     `imresize` is deprecated!
    `imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
    Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.
  frame = imresize(frame, (60, 80))


Preparing dev dataset ...
Preparing test dataset ...


In [ ]:

print("Loading dataset ..")
train_videos = pickle.load(open("/content/drive/MyDrive/public_assault/pixel_feature/train.p", "rb"))
validation_videos = pickle.load(open("/content/drive/MyDrive/public_assault/pixel_feature/dev.p", "rb"))
test_videos = pickle.load(open("/content/drive/MyDrive/public_assault/pixel_feature/test.p", "rb"))


Loading dataset ..


In [ ]:
def extract_optflow(dataset="train"):
    if dataset == "train":
        ID = Train_Id
    elif dataset == "dev":
        ID = Validation_Id
    else:
        ID = Test_Id

    # Setup parameters for optical flow.
    farneback_params = dict(
        winsize=20, iterations=1,
        flags=cv2.OPTFLOW_FARNEBACK_GAUSSIAN, levels=1,
        pyr_scale=0.5, poly_n=5, poly_sigma=1.1, flow=None)

    frames_idx = clean_sequence_file()

    data = []

    for category in CATEGORIES:
        if category == "violence":
            folder_path =os.path.join("/content/", category)
            filenames = sorted(os.listdir(folder_path))

            for filename in filenames:
                if filename == ".ipynb_checkpoints":
                    continue
                filepath ="/content/"+category+"/"+filename

                vid = imageio.get_reader(filepath, "ffmpeg")

                flow_x = []
                flow_y = []

                prev_frame = None
                for i, frame in enumerate(vid):

                    frame = cv2.resize(frame, (120, 160))
                    # Convert to grayscale.
                    frame = Image.fromarray(np.array(frame))
                    frame = frame.convert("L")
                    frame = np.array(frame.getdata(),
                                    dtype=np.uint8).reshape((IMG_W_ORIGINAL, IMG_H_ORIGINAL))
                    frame = cv2.resize(frame, (IMG_W, IMG_H))

                    if prev_frame is not None:
                        # Calculate optical flow.
                        flows = cv2.calcOpticalFlowFarneback(prev_frame, frame,
                                                            **farneback_params)
                        print(flows.shape)
                        # Subsampling optical flow by half (due to memory usage problems).
                        subsampled_x = np.zeros((IMG_W//2, IMG_H//2), dtype=np.float32)
                        subsampled_y = np.zeros((IMG_W//2, IMG_H//2), dtype=np.float32)
                        for r in range(IMG_W//2):
                            for c in range(IMG_H//2):
                                subsampled_x[r, c] = flows[r*2, c*2, 0]
                                subsampled_y[r, c] = flows[r*2, c*2, 1]

                        flow_x.append(subsampled_x)
                        flow_y.append(subsampled_y)

                    prev_frame = frame

                data.append({
                    "filename": filename,
                    "category": category,
                    "flow_x": flow_x,
                    "flow_y": flow_y
                })
        else:
            folder_path =os.path.join("/content/", category)
            filenames = sorted(os.listdir(folder_path))

            for filename in filenames:
                filepath ="/content/"+category+"/"+filename

                person_id = int(filename.split("_")[0][6:])
                if person_id not in ID:
                    continue

                vid = imageio.get_reader(filepath, "ffmpeg")

                flow_x = []
                flow_y = []

                prev_frame = None
                for i, frame in enumerate(vid):
                    ok = False
                    for seg in frames_idx[filename]:
                        if i >= seg[0] and i <= seg[1]:
                            ok = True
                            break
                    if not ok:
                        continue

                    # Convert to grayscale.
                    frame = Image.fromarray(np.array(frame))
                    frame = frame.convert("L")
                    frame = np.array(frame.getdata(),
                                    dtype=np.uint8).reshape((IMG_W_ORIGINAL, IMG_H_ORIGINAL))
                    frame = cv2.resize(frame, (IMG_W, IMG_H))

                    if prev_frame is not None:
                        # Calculate optical flow.
                        flows = cv2.calcOpticalFlowFarneback(prev_frame, frame,
                                                            **farneback_params)

                        # Subsampling optical flow by half (due to memory usage problems).
                        subsampled_x = np.zeros((IMG_W//2, IMG_H//2), dtype=np.float32)
                        subsampled_y = np.zeros((IMG_W//2, IMG_H//2), dtype=np.float32)
                        for r in range(IMG_W//2):
                            for c in range(IMG_H//2):
                                subsampled_x[r, c] = flows[r*2, c*2, 0]
                                subsampled_y[r, c] = flows[r*2, c*2, 1]

                        flow_x.append(subsampled_x)
                        flow_y.append(subsampled_y)

                    prev_frame = frame

                data.append({
                    "filename": filename,
                    "category": category,
                    "flow_x": flow_x,
                    "flow_y": flow_y
                })


    pickle.dump(data, open("/content/drive/MyDrive/public_assault/temporal_feature/%s_flow.p" % dataset, "wb"))

In [ ]:
print("Extracting train optflow ...")
extract_optflow(dataset="train")
print("Extracting dev optflow ...")
extract_optflow(dataset="dev")
print("Extracting test optflow ...")
extract_optflow(dataset="test")